## Classification on MNIST data set

In [1]:
# import packages
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# load data from tensorflow datasets
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test'] # define train and test data sets

In [3]:
# mnist has only train and test sets => devide train in train and validation
#define how much data will be used for training, validation and test
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples # define 10% of training data as validation
num_validation_samples = tf.cast(num_validation_samples, tf.int64) # tf.cast(number, format) make sure number is an integer
num_train_samples = mnist_info.splits['train'].num_examples - num_validation_samples # num of training samples left
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64) # number of test samples

In [4]:
# scale the data, so instaed of 0-255(255=> max grey scale pixel color strength) values it has 0-1
# we pass mnist data that is a tuple of inputs(image) and outputs(label)
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map
def scale_f(image, label):
    image = tf.cast(image, tf.float32) 
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale_f) # pass custom scale function to tf.dataset.map function
scaled_test_data = mnist_test.map(scale_f)

In [5]:
# shuffle the data to be sure order is random
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
# devide shaffled train dataset into train and validation data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [6]:
# to use badtch gradient descent we need to define batches of data
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE) # because train data will be backpropagated
validation_data = validation_data.repeat().batch(num_validation_samples) # batch size = whole data set, only forward propagation will be done
test_data = scaled_test_data.batch(num_test_samples) # whole data set batch, no backpropagation will be done on test data

# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
# todo: figure out this magic that does not work
# validation_inputs, validation_targets = next(iter(validation_data))

In [7]:
# outline the model

input_size = 784 # 28x28 pixels (each image is 28x28 pixels => 784)
output_size = 10 # output is numbers between 0 and 9

# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layers_1 = 100
hidden_layers_2 = 65 
hidden_layers_3 = 40
# define how the model will look like
model = tf.keras.Sequential([
    
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # flatten (28x28x1,) tensor to (784,) vector => for feed forward network
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    
    # tf.keras.layers.Dense => output = activation(dot(input, weight) + bias)
    # arguments : hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layers_1, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layers_2, activation='selu'), # 2nd hidden layer
    tf.keras.layers.Dense(hidden_layers_3, activation='relu'), # 2nd hidden layer
    
    # softmax activation function to convert values to probabilities
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

In [8]:
# compile the model
# define the optimizer and the loss function
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# train the model
NUM_EPOCHS = 5 # max number of epochs

# we fit the model,
# provide training data, the total number of epochs, the validation data
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=validation_data, validation_steps=1, verbose =2)

Epoch 1/5
540/540 - 7s - loss: 0.3336 - accuracy: 0.9016 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 8s - loss: 0.1307 - accuracy: 0.9605 - val_loss: 0.1170 - val_accuracy: 0.9657
Epoch 3/5
540/540 - 9s - loss: 0.0940 - accuracy: 0.9709 - val_loss: 0.0862 - val_accuracy: 0.9752
Epoch 4/5
540/540 - 8s - loss: 0.0710 - accuracy: 0.9777 - val_loss: 0.0834 - val_accuracy: 0.9750
Epoch 5/5
540/540 - 8s - loss: 0.0567 - accuracy: 0.9821 - val_loss: 0.0620 - val_accuracy: 0.9820


In [11]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

      1/Unknown - 1s 1s/step - loss: 0.0879 - accuracy: 0.97 - 1s 1s/step - loss: 0.0879 - accuracy: 0.9745Test loss: 0.09. Test accuracy: 97.45%


In [12]:
print (test_loss)

0.08789727836847305


In [13]:
print (test_accuracy)

0.9745
